Fruit Freshness Detection Code (.py)

In [ ]:
from __future__ import division
import cv2
import numpy as np
import serial  # For Arduino communication
import time
from copy import deepcopy

# Arduino Serial Communication
ser = serial.Serial('COM3', 9600)  # Adjust COM3 to the correct port for Arduino

# Setup kernels for morphological operations
kernelOpen = np.ones((5, 5))
kernelClose = np.ones((20, 20))

# Capture image from camera
camera = cv2.VideoCapture(0)

def detect_freshness(frame):
    """
    Function to detect freshness of vegetables and fruits based on color masks.
    """
    edge_img = deepcopy(frame)

    # Edge detection
    edged = cv2.Canny(edge_img, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)

    # Find contours
    _, cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour area
    if cnts:
        max_cont = max(cnts, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(max_cont)
        croppedk = frame[y:y+h, x:x+w]
        hsv = cv2.cvtColor(croppedk, cv2.COLOR_BGR2HSV)

        # Define color masks for freshness detection

        # Green color mask (typically for fresh vegetables)
        lower_green = np.array([35, 50, 50])
        upper_green = np.array([85, 255, 255])
        greenmask = cv2.inRange(hsv, lower_green, upper_green)

        # Red color mask (typically for fresh fruits)
        lower_red = np.array([0, 50, 50])
        upper_red = np.array([10, 255, 255])
        redmask1 = cv2.inRange(hsv, lower_red, upper_red)
        lower_red = np.array([170, 50, 50])
        upper_red = np.array([180, 255, 255])
        redmask2 = cv2.inRange(hsv, lower_red, upper_red)
        redmask = redmask1 + redmask2

        # Yellow color mask (typically for ripe fruits)
        lower_yellow = np.array([20, 50, 50])
        upper_yellow = np.array([30, 255, 255])
        yellowmask = cv2.inRange(hsv, lower_yellow, upper_yellow)

        # Brown color mask (for spoilage)
        lower_brown = np.array([10, 50, 50])
        upper_brown = np.array([20, 255, 255])
        brownmask = cv2.inRange(hsv, lower_brown, upper_brown)

        # Calculate the percentage of each color
        cnt_r = np.count_nonzero(redmask)
        cnt_g = np.count_nonzero(greenmask)
        cnt_y = np.count_nonzero(yellowmask)
        cnt_brown = np.count_nonzero(brownmask)

        total_area = cnt_r + cnt_g + cnt_y + cnt_brown
        if total_area == 0:
            return None

        brown_perc = cnt_brown / total_area  # Spoilage percentage
        fresh_perc = (cnt_g + cnt_r + cnt_y) / total_area  # Freshness percentage

        # Determine freshness
        if brown_perc > 0.5:
            freshness = "Spoilt"
        else:
            freshness = "Fresh"

        return freshness
    return None

while True:
    # Capture frame-by-frame
    ret, frame = camera.read()

    if not ret:
        print("Failed to grab frame")
        break

    # Detect freshness of the object
    freshness = detect_freshness(frame)

    # If freshness detected, display and act accordingly
    if freshness:
        cv2.putText(frame, f"Freshness: {freshness}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        print(f"Detected Freshness: {freshness}")

        # Send signal to Arduino
        if freshness == "Spoilt":
            ser.write(b'1')  # Send signal to Arduino to push the spoilt item off the belt
        else:
            ser.write(b'0')  # Send signal to let fresh item pass

    # Display the frame
    cv2.imshow('Freshness Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
camera.release()
cv2.destroyAllWindows()
ser.close()  # Close serial communication

Arduino Code for Servo Control - standard (.cpp)

In [ ]:
#include <Servo.h>

Servo myServo;
int servoPin = 9;  // Pin connected to the servo

void setup() {
  Serial.begin(9600);    // Initialize serial communication at 9600 baud rate
  myServo.attach(servoPin);  // Attach the servo to pin 9
}

void loop() {
  if (Serial.available()) {
    char command = Serial.read();  // Read the incoming data from Python

    if (command == '1') {
      // Rotate servo to push the spoilt product off the belt
      myServo.write(90);   // Move servo to 90 degrees
      delay(1000);         // Hold position for 1 second
      myServo.write(0);    // Move back to 0 degrees
      delay(1000);         // Wait before the next check
    }
    else if (command == '0') {
      // Do nothing, let the fresh product pass
      myServo.write(0);  // Ensure the servo stays in the initial position
    }
  }
}


Product Defectiveness Detection Code (.py)

In [ ]:
import torch
import cv2
import serial  # For Arduino communication
import time

# Initialize the serial communication with Arduino
ser = serial.Serial('COM3', 9600)  # Replace COM3 with your Arduino's port

# Load the pre-trained YOLOv5 model from the RIDAC repo
model = torch.hub.load('ultralytics/yolov5', 'custom', path='path_to_trained_model.pt')  # Path to your trained model

# Load a test video or use camera feed for real-time defect detection
cap = cv2.VideoCapture(0)  # Use a webcam or a video file

def activate_servo():
    """
    Function to activate the servo motor via Arduino.
    """
    ser.write(b'1')  # Send signal to Arduino to push defective item off the conveyor belt
    time.sleep(1)    # Wait for the servo to complete its action
    ser.write(b'0')  # Reset the servo to the original position

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform defect detection on the frame using YOLOv5 model
    results = model(frame)

    # Process the results (defects are usually classified as a specific class, e.g., 'defective')
    detected_items = results.xyxy[0]  # Get detected items

    for *box, conf, cls in detected_items:
        label = model.names[int(cls)]  # Class label (e.g., 'defective', 'non_defective')
        confidence = conf.item()

        # Check if the detected item is defective
        if label == 'defective' and confidence > 0.5:  # Adjust confidence threshold if needed
            print("Defective item detected with confidence: {:.2f}".format(confidence))

            # Activate the servo motor to remove the defective item
            activate_servo()

        # Draw bounding boxes around the detected objects
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame with detection
    cv2.imshow('Defect Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close all windows
cap.release()
cv2.destroyAllWindows()
ser.close()  # Close the serial communication

Arduino Code for Servo Control - generic (.cpp)

In [ ]:
#include <Servo.h>

Servo myServo;
int servoPin = 9;  // Pin connected to the servo motor

void setup() {
  Serial.begin(9600);    // Start serial communication at 9600 baud rate
  myServo.attach(servoPin);  // Attach the servo to pin 9
}

void loop() {
  if (Serial.available()) {
    char command = Serial.read();  // Read the incoming data from Python

    if (command == '1') {
      // Rotate servo to push the defective item off the conveyor belt
      myServo.write(90);   // Move servo to 90 degrees
      delay(1000);         // Hold position for 1 second
      myServo.write(0);    // Move back to 0 degrees
      delay(1000);         // Wait before the next check
    }
    else if (command == '0') {
      // Do nothing, keep the servo in the original position
      myServo.write(0);  // Ensure the servo stays at 0 degrees
    }
  }
}